<h4 align="right">Orestis Charalambous</h4> 
<h4 align="right">Elif Naz Bozkurt</h4> 
<h1><center><font size="50">Task 1</font></center></h1>

## Characters

The environment is a NxN array of characters. 
Each cell of this environment can have the following values:
- 0 : ' '      Empty Space
- 1 : 'X'      Walls
- 2 : 'A'      Agent
- 3 : 'E'      Enemy
- 4 : 'C'      Coin
- 5 : 'P'      Princess

## The grid

The grid is created by division of the castle environment by starting with two borders, one horizontally and one vertically. This way the castle is divided into 4 parts. Then it randomly chooses 3 of the 4 parts to add an empty space for the agent to be able to go through. The initial plan was for our game to be a normal maze with borders left and right or up and down as the agent moves but we run into problems as we wanted to add enemys that blocked the agent from going through. Thus, we created the maze more spacious.

## The game

The agent starts in a random position at the bottom of the maze and tries to find it's way towards the princess. The princess is randomly positioned in the top of the maze.

At each step:
- The agent decides on an action to move to(up, down, left, right)
- The action is sent to the environment and returns observations, rewards and a boolean that indicates whether the environment terminated.

The environment terminates if the agent reaches the princess, it collides with the enemy or run out of steps.

## Observations

Returns a tuple of relative coordinates between the agent and the princess(The exit).

## Rewards

When performing an action, the agent receives a reward depending on the cell it ends up on:
- If the agent collides with a Coin, it receives a reward of 150.
- If the agent collides with an Enemy, it receives a reward of -500 and the environment resets with the agent in the original position.
- If the agent collides with the Princess, it receives a reward of 300 and the environment resets.
- At each action, the agent receives a reward penalty of -1.
- If the agent tries to go through a wall it receives a reward of -10.

# Defining the Environment

**Note**: Running the below cells will demonstrate the enviroment for 15 seconds.

## Importing Libraries

In [1]:
import turtle         # Turtle graphics
import random
import numpy as np
import time


SIZE = 24             # Size of Maze

In [2]:
wn = turtle.Screen()                      # Define turtle screen.
wn.bgcolor("black")                       # Set Background color.
wn.title("Mission Impossible")            # Title of Maze.
wn.setup(SIZE*30, SIZE*30)                # Setup dimensions of working window.
wn.tracer(0)                              # Instant display.


# Assign images in a list
images = ["wall.gif", "enemy_down.gif", "enemy_up.gif", "enemy_left.gif", "enemy_right.gif",
          "gold.gif", "player_down.gif", "princess.gif", "player_left.gif", "player_up.gif", "player_right.gif"]

for image in images:
    turtle.register_shape(image)           # Register graphics in turtle.
    
    
    
############## Creating Classes for turtle graphics #############
            
    
    
class Pen(turtle.Turtle):                  # Define the maze before graphics.
    def __init__(self):
        turtle.Turtle.__init__(self)
        self.shape("square")               # Turtle shape.
        self.color("white")                # Colour of turtle.
        self.penup()                       # Lift up pen so it does not leave a trail.
        self.speed('fastest')              # Set speed that the maze in written on the screen.


class Player(turtle.Turtle):               # Define the Agent.
    def __init__(self):
        turtle.Turtle.__init__(self)
        self.shape("player_down.gif")      # Image of Agent.
        self.color("blue")                 # Initial colour before graphics.
        self.penup()
        self.speed('fastest')              # Drawing speed.

    def go_up(self):                       # Calculate the next position to move to
        move_to_x = player.xcor()
        move_to_y = player.ycor() + 24
        self.shape("player_up.gif")

        return move_to_x, move_to_y

    def go_down(self):                     # Calculate the next position to move to.
        move_to_x = player.xcor()
        move_to_y = player.ycor() - 24
        self.shape("player_down.gif")

        return move_to_x, move_to_y

    def go_left(self):                     # Calculate the next position to move to.
        move_to_x = player.xcor() - 24
        move_to_y = player.ycor()
        self.shape("player_left.gif")

        return move_to_x, move_to_y

    def go_right(self):                    # Calculate the next position to move to.
        move_to_x = player.xcor() + 24
        move_to_y = player.ycor()
        self.shape("player_right.gif")

        return move_to_x, move_to_y

    def is_collision(self, other):
        if player.distance(other) < 5:     # Calculate if objects collide.
            return True
        else:
            return False


class Princess(turtle.Turtle):          # Define Princess.
    def __init__(self):
        turtle.Turtle.__init__(self)
        self.shape("princess.gif")      # Image of princess.
        self.color("yellow")            # Initial colour before graphics.
        self.penup()
        self.gold = 300                 # Used as a reward when agent collides with princess.
        self.speed("fastest")           # Drawing speed.


class Coin(turtle.Turtle):          # Define Coin.
    def __init__(self):
        turtle.Turtle.__init__(self)
        self.shape("gold.gif")          # Image of Coin.
        self.color("gold")              # Initial colour before graphics.
        self.penup()
        self.speed("fastest")           # Drawing speed.
        self.gold = 150                 # Used as a reward when agent collides with Coin.

    def destroy(self):
        self.goto(1000, 1000)           # Moves Coin outside of the screen.


class Enemy(turtle.Turtle):             # Define Enemy.
    def __init__(self):
        turtle.Turtle.__init__(self)
        self.shape("enemy_down.gif")    # Image of enemy.
        self.color("red")               # Initial colour before graphics.
        self.penup()
        self.speed("fastest")           # Drawing speed.
        self.gold = 500                 # Used as a penalty when agent collides with enemy.
        self.direction = random.choice(["up", "down", "left", "right"])  # Used when enemy moves automaticaly.

    def move(self):                     # Coordinates are in pixels.
        if self.direction == "up":
            dx = 0
            dy = 24
            self.shape("enemy_up.gif")
        elif self.direction == "down":
            dx = 0
            dy = -24
            self.shape("enemy_down.gif")
        elif self.direction == "left":
            dx = -24
            dy = 0
            self.shape("enemy_left.gif")
        elif self.direction == "right":
            dx = 24
            dy = 0
            self.shape("enemy_right.gif")
        else:
            dx = 0
            dy = 0

        # Calculate the next position to move to.
        move_to_x = self.xcor() + dx
        move_to_y = self.ycor() + dy

        if (move_to_x, move_to_y) not in walls:     # Check if the next position is a wall.
            self.goto(move_to_x, move_to_y)
        else:
            self.direction = random.choice(["up", "down", "left", "right"])  # Choose a different direction.

        turtle.ontimer(self.move, t=random.randint(100, 300))                # Set timer for the next move.

        
        
############# Create random maze #############      
             
        
        
def maze(size):                 
    map_characters = {0: ' ',   # Empty Space
                      1: 'X',   # Walls
                      2: 'A',   # Agent
                      3: 'E',   # Enemy
                      4: 'C',   # Coins
                      5: 'P'}   # Princess

    grid = []
    for r in range(size):                       # Creating empty space Grid.
        grid.append([])
        for c in range(size):
            grid[r].append(map_characters[0])

    for r in range(len(grid)):                  # Creates the first and last column border.
        grid[r][0] = map_characters[1]
        grid[r][len(grid[r]) - 1] = map_characters[1]

    for c in range(1, len(grid[0]) - 1):        # Creates the first and last row border.
        grid[0][c] = map_characters[1]
        grid[len(grid) - 1][c] = map_characters[1]

    def grid_recursion(grid_, top_border, bottom_border, first_col_border, last_col_border):

        # Figure out where to divide horizontally
        beginning_point = bottom_border + 2
        finishing_point = top_border - 1
        chosen_col = random.randrange(beginning_point, finishing_point, 2)

        # Do the division
        for column in range(first_col_border + 1, last_col_border):
            grid_[chosen_col][column] = map_characters[1]

        # Figure out where to divide vertically
        beginning_point = first_col_border + 2
        finishing_point = last_col_border - 1
        chosen_row = random.randrange(beginning_point, finishing_point, 2)

        # Do the division
        for row in range(bottom_border + 1, top_border):
            grid_[row][chosen_row] = map_characters[1]

        # Now we'll make a gap on 3 of the 4 walls.
        # Figure out which wall does NOT get a gap.
        
        new_border = random.randint(0, 3) 
        if new_border != 0:
            space = random.randrange(first_col_border + 1, chosen_row, 2)
            grid_[chosen_col][space] = map_characters[0]
            grid_[chosen_col][space + 1] = map_characters[0]

        if new_border != 1:
            space = random.randrange(chosen_row + 1, last_col_border, 2)
            grid_[chosen_col][space] = map_characters[0]
            grid_[chosen_col][space - 1] = map_characters[0]

        if new_border != 2:
            space = random.randrange(bottom_border + 1, chosen_col, 2)
            grid_[space][chosen_row] = map_characters[0]
            grid_[space][chosen_row + 1] = map_characters[0]

        if new_border != 3:
            space = random.randrange(chosen_col + 1, top_border, 2)
            grid_[space][chosen_row] = map_characters[0]
            grid_[space][chosen_row - 1] = map_characters[0]

        # If there's enough space, do a recursive call.
        if top_border > chosen_col + 3 and chosen_row > first_col_border + 3:
            grid_recursion(grid_, top_border, chosen_col, first_col_border, chosen_row)

        if top_border > chosen_col + 3 and chosen_row + 3 < last_col_border:
            grid_recursion(grid_, top_border, chosen_col, chosen_row, last_col_border)

        if bottom_border + 3 < chosen_col and chosen_row + 3 < last_col_border:
            grid_recursion(grid_, chosen_col, bottom_border, chosen_row, last_col_border)

        if bottom_border + 3 < chosen_col and chosen_row > first_col_border + 3:
            grid_recursion(grid_, chosen_col, bottom_border, first_col_border, chosen_row)

        return grid_

    recursion = grid_recursion(grid, size - 1, 0, 0, size - 1)      # Doing recursion again.

    complete_castle = []
    for i in range(1):    # Assigning random positions for Enemy, Coin, Agent and Princess.
        recursion[random.randint(2, size - 4)][random.randint(2, size - 2)] = map_characters[3]                                  
        recursion[random.randint(2, size - 3)][random.randint(2, size - 2)] = map_characters[4]
        recursion[size - 2][random.randint(1, size - 4)] = map_characters[2]                      # Agent starts at the bottom of the maze.
        recursion[1][random.randint(4, size - 2)] = map_characters[5]                            # Princess is at the top of the maze.

    for i, j in enumerate(recursion):       # Organising the list to (size) strings for easier turtle access.
        joined = ''.join(recursion[i])
        complete_castle.append(joined)

    return complete_castle                  # Returning complete maze.



########### Observation function ###########



def observations():
    obs = (player.pos() - princess.pos())
    obs = [round(num / 24) for num in list(obs)]
    obs = tuple(obs)

    return obs



############### Step function ###############



def step(action):
    if action == 0:
        move = player.go_up()            # move variable returns coordinates of next position.
    elif action == 1:
        move = player.go_down()       
    elif action == 2:
        move = player.go_left()
    elif action == 3:
        move = player.go_right()

    stop = False
    if move in walls:                    # Check if the next position is a wall.
        stop = True
    else:
        player.goto(move)                # Go to next position.

    if player.is_collision(coin):
        reward = coin.gold               # Assign coin gold to reward.
        coin.destroy()                   # Remove coin.
        done = False
    elif player.is_collision(princess):  # Reward for colliding with princess.
        reward = princess.gold
        done = True
    elif player.is_collision(enemy):     # Reward for colliding with enemy.
        reward = -enemy.gold
        done = True
    elif stop:                           # Reward for colliding at a wall.
        reward = -10
        done = False
    else:                             
        reward = -1                      # Moving penalty reward.
        done = False

    obs = observations()
        
    return obs, reward, done



################# Reset function ############



def reset(game):                   
    global castle
    setup_maze(game, SIZE)         # Setups maze.
    pen.penup()
    state = observations()         # Returns current state.
    return state



############ Create level setup function ##########



def setup_maze(level, size):
    global walls                              # Used global walls so it creates an empty walls list each time a new level is generated in each episode.
    walls = []                                 
    for y in range(len(level)):               # Select each line in the level.
        for x in range(len(level[y])):        # Select each character in the level.
            character = level[y][x]           # Assigning the above character to a variable character.
            screen_x = -(((size-1)*24)/2) + (x * 24)      # Calculating X and Y coordinates for starting position of characters according to pixels.
            screen_y = ((size-1)*24)/2 - (y * 24)          

            if character == "X":
                pen.goto(screen_x, screen_y)        # If character is "X", assign it as a wall.
                pen.shape("wall.gif")
                pen.stamp()                         # Leave an impression on the screen.
                walls.append((screen_x, screen_y))  # Add coordinate to wall list.

            if character == "A":                    # If character is "A", assign it as an Agent.
                player.goto(screen_x, screen_y)

            if character == "P":                    # If character is "P", assign it as a Princess.
                princess.goto(screen_x, screen_y)

            if character == "C":
                coin.goto(screen_x, screen_y)       # If character is "C", assign it as a Coin.

            if character == "E":
                enemy.goto(screen_x, screen_y)      # If character is "E", assign it as an Enemy.
                

# Create class instances
pen = Pen()
player = Player()
princess = Princess()
enemy = Enemy()
coin = Coin()


# Create list instance
walls = []

wn.tracer(0)                        # Turn off screen updates

# Sets up the level
setup_maze(maze(SIZE), SIZE)

turtle.ontimer(enemy.move, t=250)  # Initial movement of enemy.(used when enemy moves automatically)

#This function is used just to demonstrate the game.
for i in range(30):                 # Run for 15 seconds
    wn.update()                     # Update Screen
    step(np.random.randint(0,4))
    time.sleep(0.5)

wn.bye()                            # Closes turtle screen.